# **Data Cleaning**

## Objectives

* Import the Dataset
* Analyze the necessary steps for data cleaning

## Inputs

* The dataset is located at: outputs/datasets/collection in the root level directory

## Outputs

* Cleaned dataset, ready for correlation studies in order to answer business requirements



---

# Change working directory

* We are assuming you will store the notebooks in a subfolder, therefore when running the notebook in the editor, you will need to change the working directory

We need to change the working directory from its current folder to its parent folder
* We access the current directory with os.getcwd()

In [1]:
import os
current_dir = os.getcwd()
current_dir

'/workspaces/Fabrizio-Project-Five/jupyter_notebooks'

We want to make the parent of the current directory the new current directory
* os.path.dirname() gets the parent directory
* os.chir() defines the new current directory

In [2]:
os.chdir(os.path.dirname(current_dir))
print("You set a new current directory")

You set a new current directory


Confirm the new current directory

In [3]:
current_dir = os.getcwd()
current_dir

'/workspaces/Fabrizio-Project-Five'

# Analysis of the dataset

In this section we will take a look at all the missing data in the dataset

In [4]:
import pandas as pd
df = pd.read_csv('inputs/datasets/raw/house-price-20211124T154130Z-001/house-price/house_prices_records.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 24 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   1stFlrSF       1460 non-null   int64  
 1   2ndFlrSF       1374 non-null   float64
 2   BedroomAbvGr   1361 non-null   float64
 3   BsmtExposure   1460 non-null   object 
 4   BsmtFinSF1     1460 non-null   int64  
 5   BsmtFinType1   1346 non-null   object 
 6   BsmtUnfSF      1460 non-null   int64  
 7   EnclosedPorch  136 non-null    float64
 8   GarageArea     1460 non-null   int64  
 9   GarageFinish   1298 non-null   object 
 10  GarageYrBlt    1379 non-null   float64
 11  GrLivArea      1460 non-null   int64  
 12  KitchenQual    1460 non-null   object 
 13  LotArea        1460 non-null   int64  
 14  LotFrontage    1201 non-null   float64
 15  MasVnrArea     1452 non-null   float64
 16  OpenPorchSF    1460 non-null   int64  
 17  OverallCond    1460 non-null   int64  
 18  OverallQ

Now the features with missing values are:
* 2ndFlrSF
* BedroomAbvGr
* BsmtFinType1
* EnclosedPorch
* GarageFinish
* GarageYrBlt
* LotFrontage
* MasVnrArea
* WoodDeckSF

 Of these features only 2 are categorical (BsmtFinType1, GarageFinish). In order to transform these two features and the other categorical features into numbers we will have to use some feature engineering techniques. These steps will be highlighted later in another notebook since ML algorithms and correlation techniques need numerical data in order to work. 

 Now we will look at our missing values from features with numerical data type. 

- 2ndFlrSF stands for the square feet of surface found at the 2nd floor. Given the fact that this dataset is all from houses in the same area we can fill the missing data with a mean value found for that column.
- BedroomAbvGr stands for bedroom above grade and it's a score that spans 0 to 8. Here too we can substitute the missing values with a mean score in order to keep the distribution unaffected.
- Enclosed Porch is a square feet range of surface area. Since many of the values are 0 and only few have a square feet value above 0 we can infer that those with 0 are houses with no enclosed porch. We will fill the missing values with 0.
- GarageYrBlt stands for the year during which the house garage has been built. In this case the missing data is present due to the fact that the house in question has no garage. It would be inprudent to fill the missing values with the mean of the year or other imputation methods. In this case we will drop the rows with the missing data.
- Lot frontage stands for the amount of linear feet of the property in contact with the road. For all the missing values we will impute the mean value of the column.
- MasVnrArea. This column refers to the masonry veneer area of the prorperty. Since the vast majority of data points for this feature are 0 we will fill the few (just 8 data points) empty slots with 0.
- WoodDeckSF. This feature refers to the surface area of the wooden deck in square feet. The number of missing data in this feature is greater than in any other but we can safely assume that this is due to a lack of a wooden deck in the house taken into consideration. For this column we can simply impute 0 for all the missing values.

---

# Data cleaning steps

In this section we will apply the cleaning steps as highlighted in the previous paragraph.
A quick summary of the steps we'll take:
- 2ndFlrSF: impute mean value for the missing data
- BedroomAbvGr: impute mean value for the missing data
- Enclosed Porch: impute value of 0 for the missing data
- GarageYrBlt: drop the rows with missing data
- LotFrontage: impute mean value for the missing data
- MasVnrArea: impute value of 0 for the missing data
- WoodDeckSF: impute value of 0 for the missing data

In [43]:
# First we drop the rows with missing data from 'GarageYrBlt'
df = df.dropna(subset='GarageYrBlt')

"""
This function will iterate over all the numerical features with missing variables and impute the missing data according 
to what we specified in the previous cells
"""
num_vars_with_miss = ['2ndFlrSF', 'BedroomAbvGr', 'EnclosedPorch', 'LotFrontage', 'MasVnrArea', 'WoodDeckSF']
def impute_missing_values(vars_to_impute, df):
    for col in vars_to_impute:
        if col in ['EnclosedPorch', 'MasVnrArea', 'WoodDeckSF']:
            df[col] = df[col].fillna(0)
        else:
            if col in ['2ndFlrSF', 'BedroomAbvGr', 'LotFrontage',]:
                df[col] = df[col].fillna(df[col].mean())
    return df
            
impute_missing_values(num_vars_with_miss, df)

,1stFlrSF,2ndFlrSF,BedroomAbvGr,BsmtExposure,BsmtFinSF1,BsmtFinType1,BsmtUnfSF,EnclosedPorch,GarageArea,GarageFinish,...,LotFrontage,MasVnrArea,OpenPorchSF,OverallCond,OverallQual,TotalBsmtSF,WoodDeckSF,YearBuilt,YearRemodAdd,SalePrice
0,856,854.000000,3.000000,No,706,GLQ,150,0.0,548,RFn,...,65.0,196.0,61,5,7,856,0.0,2003,2003,208500
1,1262,0.000000,3.000000,Gd,978,ALQ,284,0.0,460,RFn,...,80.0,0.0,0,8,6,1262,0.0,1976,1976,181500
2,920,866.000000,3.000000,Mn,486,GLQ,434,0.0,608,RFn,...,68.0,162.0,42,5,7,920,0.0,2001,2002,223500
3,961,348.524017,2.869214,No,216,ALQ,540,0.0,642,Unf,...,60.0,0.0,35,5,7,756,0.0,1915,1970,140000
4,1145,348.524017,4.000000,Av,655,GLQ,490,0.0,836,RFn,...,84.0,350.0,84,5,8,1145,0.0,2000,2000,250000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,953,694.000000,3.000000,No,0,Unf,953,0.0,460,RFn,...,62.0,0.0,40,5,6,953,0.0,1999,2000,175000
1456,2073,0.000000,2.869214,No,790,ALQ,589,0.0,500,Unf,...,85.0,119.0,0,6,6,1542,0.0,1978,1988,210000
1457,1188,1152.000000,4.000000,No,275,GLQ,877,0.0,252,RFn,...,66.0,0.0,60,9,7,1152,0.0,1941,2006,266500
1458,1078,0.000000,2.000000,Mn,49,NaN,0,112.0,240,Unf,...,68.0,0.0,0,6,5,1078,0.0,1950,1996,142125


Now that we have filled the missing values for the numerical features we can check the dataset to ascertain the current status after the changes we made

In [44]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1379 entries, 0 to 1459
Data columns (total 24 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   1stFlrSF       1379 non-null   int64  
 1   2ndFlrSF       1379 non-null   float64
 2   BedroomAbvGr   1379 non-null   float64
 3   BsmtExposure   1379 non-null   object 
 4   BsmtFinSF1     1379 non-null   int64  
 5   BsmtFinType1   1275 non-null   object 
 6   BsmtUnfSF      1379 non-null   int64  
 7   EnclosedPorch  1379 non-null   float64
 8   GarageArea     1379 non-null   int64  
 9   GarageFinish   1225 non-null   object 
 10  GarageYrBlt    1379 non-null   float64
 11  GrLivArea      1379 non-null   int64  
 12  KitchenQual    1379 non-null   object 
 13  LotArea        1379 non-null   int64  
 14  LotFrontage    1379 non-null   float64
 15  MasVnrArea     1379 non-null   float64
 16  OpenPorchSF    1379 non-null   int64  
 17  OverallCond    1379 non-null   int64  
 18  OverallQ

Now let's tackle the object features with missing data. As we know from our previous analysis these features are: Garage Finish and BsmtFinType1

-  Garage Finish: This feature collects data on the status of the garage which can be:
    - Finished
    - Rough finished
    - Unfinished
    - None
   
For the missing data here we can impute 'Unfinished' because where None is present we see that the feature 'Garage Area' also shows 0, confirming the absence of a garage.

---
- BsmtFinType1: This feature referes to the state of the basement and it has the following possible values:
    - Good Living Quarters
    - Average Living Quarters
    - Below Average Living Quarters
    - Average Rec Room
    - Low Quality
    - Unfinished
    - None
     
For the missing data we can fill with 'Unfinished' as well since the missing data corresponds with either 0 or a numerical value and 'Unfinished' is also the most common value fo this feature.

0       2003.0
1       1976.0
2       2001.0
3       1998.0
4       2000.0
         ...  
1455    1999.0
1456    1978.0
1457    1941.0
1458    1950.0
1459    1965.0
Name: GarageYrBlt, Length: 1379, dtype: float64

---

# Push files to Repo

* If you do not need to push files to Repo, you may replace this section with "Conclusions and Next Steps" and state your conclusions and next steps.

In [ ]:
import os
try:
  # create here your folder
  # os.makedirs(name='')
except Exception as e:
  print(e)
